# **IMPORT FILE**

In [1]:
#import library yang dibutuhkan
import cv2
import matplotlib.pyplot as plt
import os
import zipfile
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping

# **EKSTRAK DATASET ZIP**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#ekstrak file zip berisi dataset
local_zip = '/content/drive/My Drive/Capstone/archive.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

# **ATUR ALUR PATH**

In [4]:
trainpath = '/content/train/'
valpath = '/content/val/'
testpath = '/content/test/'

In [5]:
print ("train class", len(trainpath))
print ("validation class", len(valpath))
print ("test class", len(testpath))

train class 15
validation class 13
test class 14


In [6]:
# Fungsi untuk memuat dan menampilkan gambar
def show_images(image_paths, titles):
    plt.figure(figsize=(12, 8))
    for i in range(len(image_paths)):
        plt.subplot(1, len(image_paths), i+1)
        image = cv2.imread(image_paths[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.title(titles[i])
        plt.axis('off')
    plt.show()

In [7]:
# Mendefinisikan path ke folder
folder_path = '/content/val'

# Menghapus file tertentu dalam folder
file_to_delete = 'classname.txt'
file_path = os.path.join(folder_path, file_to_delete)
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"File '{file_to_delete}' telah dihapus.")
else:
    print(f"File '{file_to_delete}' tidak ditemukan.")


File 'classname.txt' telah dihapus.


In [8]:
# Mendefinisikan path ke folder
folder_path = '/content/train'

# Menghapus file tertentu dalam folder
file_to_delete = 'classname.txt'
file_path = os.path.join(folder_path, file_to_delete)
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"File '{file_to_delete}' telah dihapus.")
else:
    print(f"File '{file_to_delete}' tidak ditemukan.")


File 'classname.txt' telah dihapus.


In [ ]:
# Lokasi folder yang berisi dataset gambar multiclass
dataset_folder = '/content/test'

# Daftar kelas atau kategori dalam dataset
classes = os.listdir(dataset_folder)

# Ambil beberapa contoh gambar dari setiap kelas
num_examples_per_class = 3
for cls in classes:
    class_folder = os.path.join(dataset_folder, cls)
    image_files = os.listdir(class_folder)[:num_examples_per_class]
    image_paths = [os.path.join(class_folder, img) for img in image_files]
    titles = [cls] * num_examples_per_class
    show_images(image_paths, titles)

In [ ]:
# Lokasi folder yang berisi dataset gambar multiclass
dataset_folder = '/content/train'

# Daftar kelas atau kategori dalam dataset
classes = os.listdir(dataset_folder)

# Ambil beberapa contoh gambar dari setiap kelas
num_examples_per_class = 3
for cls in classes:
    class_folder = os.path.join(dataset_folder, cls)
    image_files = os.listdir(class_folder)[:num_examples_per_class]
    image_paths = [os.path.join(class_folder, img) for img in image_files]
    titles = [cls] * num_examples_per_class
    show_images(image_paths, titles)

In [ ]:
# Lokasi folder yang berisi dataset gambar multiclass
dataset_folder = '/content/val'

# Daftar kelas atau kategori dalam dataset
classes = os.listdir(dataset_folder)

# Ambil beberapa contoh gambar dari setiap kelas
num_examples_per_class = 3
for cls in classes:
    class_folder = os.path.join(dataset_folder, cls)
    image_files = os.listdir(class_folder)[:num_examples_per_class]
    image_paths = [os.path.join(class_folder, img) for img in image_files]
    titles = [cls] * num_examples_per_class
    show_images(image_paths, titles)

In [ ]:
# Data Generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,           # Normalisasi nilai pixel menjadi rentang 0-1
    shear_range=0.2,          # Augmentasi data dengan shear
    zoom_range=0.2,           # Augmentasi data dengan zoom
    horizontal_flip=True,     # Augmentasi data dengan flip horizontal
    validation_split=0.2)     # Memisahkan sebagian data untuk validasi

test_val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
# Direktori Data
data_train = trainpath
data_val = valpath
data_test = testpath

In [ ]:
# Generator untuk data training
train_generator = train_datagen.flow_from_directory(
    data_train,
    target_size=(224, 224),  # Ukuran gambar sesuai dengan input MobileNetV2
    batch_size=32,
    class_mode='categorical'
)

# Generator untuk data validasi
validation_generator = test_val_datagen.flow_from_directory(
    data_val,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Generator untuk data testing
test_generator = test_val_datagen.flow_from_directory(
    data_test,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 40000 images belonging to 100 classes.
Found 5000 images belonging to 100 classes.
Found 5000 images belonging to 100 classes.


# **Membuat Model**

In [ ]:
# Load MobileNetV2 tanpa lapisan top (head)
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
    )

# Tambahkan lapisan custom di atas model pre-trained
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')  # Jumlah kelas sesuai dataset
])

# Kompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:

# Callback untuk menghentikan pelatihan jika akurasi validasi mencapai 99%
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    verbose=1,
    mode='max',
    restore_best_weights=True,
    min_delta=0.001,
)

# **Training Model**

In [ ]:
# Melatih model
history = model.fit(
    train_generator,
    epochs=50,  # Tetapkan nilai yang cukup tinggi untuk memberikan kesempatan EarlyStopping bekerja
    validation_data=validation_generator
)

# **Saving Model**

In [ ]:
# Saving the model
model.save('model.h5')

# Saving to drive
model.save('/content/drive/My Drive/Capstone/model.h5')

# **Model Evaluation**

In [ ]:
# Accuracy Plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'],
           loc = 'upperleft')
plt.show()

In [ ]:
# Loss Plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'],
           loc = 'upperleft')
plt.show()